In [1]:
import feedparser, requests
import streamlit as st
from utils.api_keys import fetch_api_key
import os

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = fetch_api_key("langsmith", requires_pass=False)
os.environ["MISTRAL_API_KEY"] = fetch_api_key("mistral", requires_pass=False)

Key loading successful.
Key loading successful.


In [3]:
import os
from langchain_core.tools import tool

In [4]:
class ArXivRetriever:
    def __init__(self, api_url="http://export.arxiv.org/api/query?"):
        self.api_url = api_url

    def fetch_papers(self, topic):
        """Fetches information about n (param: max_results) relevant research papers on given topic (param: topic) """

        max_results=5
        query_url = f"{self.api_url}search_query=all:{topic}&max_results={max_results}"
        try:
            response = requests.get(query_url)
            response.raise_for_status()
            feed = feedparser.parse(response.content)

            papers = []
            for entry in feed.entries:
                paper_id = entry.id.split("/")[-1]
                pdf_url = f"https://arxiv.org/pdf/{paper_id}.pdf"
                paper = {
                    "id": paper_id,
                    "title": entry.title,
                    "category": entry.category,
                    "authors": [author.name for author in entry.authors],
                    "author_comment": entry.get("arxiv_comment", "No comments available"),
                    "published": entry.published,
                    "summary": entry.summary,
                    "link": entry.link,
                    "pdf_url": pdf_url,
                    "journal_ref": entry.get("arxiv_journal_ref", "No journal reference")
                }
                papers.append(paper)
            return papers
        except requests.exceptions.RequestException as e:
            st.error("Error fetching papers. Please try again later.")
            return []


In [5]:
retriever = ArXivRetriever()

In [6]:
tools = [retriever.fetch_papers]

In [7]:
from langchain_core.prompts import ChatPromptTemplate

In [9]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are the research paper retriever agent, user will tell you the topic, description or any information related to research paper, you have to efficiently generate relevant topic from user's input and then return {n} relevant research papers related to that topic.:"),
        ("user","{user_requirement}"),
        ("assistant","{agent_scratchpad}")
    ]
)